In [1]:
import os
import sys
os.chdir("../")
sys.path.append("../")

import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import pickle
# from thop import profile
from collections import defaultdict
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel, BertModel, BertConfig, AutoModelForSequenceClassification
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Identical_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker
from models.Encoders.Pooling import Attention_Pooling, Average_Pooling
from models.UniLM.modeling import TuringNLRv3Model, TuringNLRv3ForSequenceClassification, relative_position_bucket
from models.UniLM.configuration_tnlrv3 import TuringNLRv3Config

from models.TwoTowerBaseModel import TwoTowerBaseModel

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.TESRec import TESRec
from models.XFormer import XFormer
 
from models.Modules.Attention import MultiheadAttention, get_attn_mask, XSoftmax
torch.set_printoptions(threshold=100000)

In [11]:
a = torch.zeros((3,4))
b = torch.ones(3,3)
index = torch.tensor([[0,1,2], [0,2,0], [1,1,0]])

In [18]:
c = torch.zeros(3)
c[[0,1,0]] += 1
c

tensor([1., 1., 0.])

In [17]:
a.index_add(index=index, source=b, dim=-1)

IndexError: index_add_(): Index is supposed to be a vector

In [2]:
from models.Modules.Synthesizer import BertSelfAttention

In [3]:
from transformers import BertConfig, BertModel
bert_config = BertConfig()
# primary bert
bert = BertModel(bert_config)
# [CLS]
bert_config.signal_length = 100
for l in bert.encoder.layer:
    l.attention.self = BertSelfAttention(bert_config)

In [5]:
bert.load_state_dict(BertModel.from_pretrained("bert-base-uncased", cache_dir=config.path + 'bert_cache/').state_dict(), strict=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_IncompatibleKeys(missing_keys=['encoder.layer.0.attention.self.dense.weight', 'encoder.layer.0.attention.self.dense.bias', 'encoder.layer.1.attention.self.dense.weight', 'encoder.layer.1.attention.self.dense.bias', 'encoder.layer.2.attention.self.dense.weight', 'encoder.layer.2.attention.self.dense.bias', 'encoder.layer.3.attention.self.dense.weight', 'encoder.layer.3.attention.self.dense.bias', 'encoder.layer.4.attention.self.dense.weight', 'encoder.layer.4.attention.self.dense.bias', 'encoder.layer.5.attention.self.dense.weight', 'encoder.layer.5.attention.self.dense.bias', 'encoder.layer.6.attention.self.dense.weight', 'encoder.layer.6.attention.self.dense.bias', 'encoder.layer.7.attention.self.dense.weight', 'encoder.layer.7.attention.self.dense.bias', 'encoder.layer.8.attention.self.dense.weight', 'encoder.layer.8.attention.self.dense.bias', 'encoder.layer.9.attention.self.dense.weight', 'encoder.layer.9.attention.self.dense.bias', 'encoder.layer.10.attention.self.dense.weight', 

In [13]:
# m = AutoModel.from_pretrained('bert-base-uncased',cache_dir=config.path + 'bert_cache/')
# m2 = AutoModel.from_pretrained('microsoft/deberta-base',cache_dir=config.path + 'bert_cache/')
# m3 = TuringNLRv3ForSequenceClassification.from_pretrained(config.unilm_path, config=TuringNLRv3Config.from_pretrained(config.unilm_config_path))
# m4 = AutoModel.from_pretrained('google/reformer-crime-and-punishment', cache_dir=config.path + "bert_cache/")
# m5 = AutoModel.from_pretrained('funnel-transformer/small-base', cache_dir=config.path + "bert_cache/")

# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")
# t3 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', cache_dir=config.path + "bert_cache/")
# t4 = AutoTokenizer.from_pretrained('google/reformer-crime-and-punishment', cache_dir=config.path + "bert_cache/")
# t5 = AutoTokenizer.from_pretrained('funnel-transformer/small-base', cache_dir=config.path + "bert_cache/")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 2002, 7459, 7861, 8270, 4667, 2015, 1012, 2054, 1037, 6616, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
inputs = t5("He loves embeddings. what a fuck", return_tensors='pt', padding="max_length", max_length=20)
tokens = t5.convert_ids_to_tokens(inputs['input_ids'][0])
output = m5(**inputs)
print(inputs, tokens, output)

{'input_ids': tensor([[ 101, 2002, 7459, 7861, 8270, 4667, 2015, 1012, 2054, 1037, 6616,  102,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])} ['<cls>', 'he', 'loves', 'em', '##bed', '##ding', '##s', '.', 'what', 'a', 'fuck', '<sep>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'] BaseModelOutput(last_hidden_state=tensor([[[-2.0142e-02, -2.6218e-02, -4.4095e-03, -2.3897e-02, -2.0012e-02,
           2.6657e-03, -1.4716e-02,  1.3300e-02,  1.0259e-02, -4.8551e-02,
          -1.4840e-02, -2.5211e-02,  1.7809e-02, -3.4188e-02,  2.9691e-03,
          -5.2072e-02, -6.2330e-03, -1.0626e-04, -1.5643e-02, -8.3762e-02,
           6.8553e-04, -3.7794e-02,  1.0386e-02, -2.2060e-02,  2.2559e-02,
          -3.3695e-02,  4.2113e-02, -3.5238e-02,  1.0513e-02, -1.6113e-03,
          -2.8745

In [ ]:
# config.reducer = 'entity'
# config.embedding = 'deberta'
# config.bert = 'longformer'
# config.seed = None
config.mode = "recall"
config.recall_type = "s"
config.scale = "large"
config.case = False

manager = Manager(config)

loaders = manager.prepare()
X1 = list(loaders[0])
# X2 = list(loaders[1])
x1 = X1[0]
# x2 = X2[0]

In [ ]:
embedding = BERT_Embedding(manager)

encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)
# encoderU = Attention_Pooling(manager)
# encoderU = Average_Pooling(manager)

reducer = Matching_Reducer(manager)
# reducer = Identical_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

model = TESRec(manager, embedding, encoderN, encoderU, reducer).to(manager.device)
# model = ESM(manager, embedding, encoderN, encoderU, reducer, ranker).to(manager.device)
# model = XFormer(manager).to(manager.device)

In [ ]:
manager.load(model, 230000)

In [ ]:
a,b,c,d,e = model.encode_user(x1)

In [ ]:
np.asarray(t.convert_ids_to_tokens(x1['his_encoded_index'][0,0])[1:])[[86,2,93]]

In [ ]:
e[0,3,35], e.shape

In [ ]:
t.convert_ids_to_tokens(x1['his_encoded_index'][0,3,[36]])

In [ ]:
print(t.convert_ids_to_tokens(x1['his_encoded_index'][0,3]))

In [ ]:
e[0,3], e.shape, e.topk(dim=-1, k=3)

In [ ]:
recall = 0
count = 0
for x in tqdm(loaders[0], ncols=120, leave=True):
    kid = model.encode_user(x)[1]
    ps_terms = x['his_encoded_index'][:, :, 1:].to(manager.device).gather(index=kid, dim=-1).view(-1).tolist()
    

In [ ]:
embedding = 32*50*2*100*768
ext_encoding = 32*50*(2*3*384*100 + 4*100*384) + 32*8*2*384*384*50
reduction = (2*100*768 + 100*math.log2(100))*32*50*2+32*50*3
bert_embedding = 32*150*768*2
bert_project = 32*150*768*2*3
bert_attn = 32*150*12*64*64*2 + 32*150*12*12*64*2
bert_intm = 32*150*768*768*2 + 32*150*768*2 + 32*150*768*3072*4
bert_pool = 32*150*768*4
bert = (bert_project + bert_attn + bert_intm) * 12 + bert_pool

total_isrec = embedding + ext_encoding + reduction + bert

embedding = 32*50*2*100*768 + 32*50*100*768*2
bert_project = 32*5000*768*2*3
bert_attn = 32*5000*12*64*64*2 + 32*5000*12*12*64*2
bert_intm = 32*5000*768*768*2 + 32*5000*768*2 + 32*5000*768*3072*4
bert_pool = 32*5000*768*4
bert = (bert_project + bert_attn + bert_intm) * 12 + bert_pool

total =  embedding + bert + 32*8*2*768*768*50

total_isrec, total